In [5]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from multiprocessing import cpu_count

from torch.utils.tensorboard import SummaryWriter

import skorch

%load_ext tensorboard
writer = SummaryWriter(log_dir='/data/runs')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    loss_sum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        writer.add_scalar('Loss/train', loss.item(), epoch*len(train_loader)+batch_idx)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': cpu_count(), 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters())

scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

0it [00:00, ?it/s]

9920512it [00:05, 1693775.03it/s]                             


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 72822.86it/s]                            
0it [00:00, ?it/s]

Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw


1654784it [00:03, 545112.79it/s]                             
0it [00:00, ?it/s]

Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw


8192it [00:00, 36421.41it/s]            


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw
Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.290535
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.361950
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.575288
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.648318
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.466340
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.603932
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.455599
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.386351
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.286836
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.268706
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.201915
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.131510
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.201852
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.205554
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.244557
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.592395
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.080515
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.220795
Train Epoch: 1 [11520/60000 (19%)]	Lo